# CS 195: Natural Language Processing
## Reinforcement Learning from Human Feedback

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ericmanley/f23-CS195NLP/blob/main/F7_4_RLHF.ipynb)

## Reference

Illustrating Reinforcement Learning from Human Feedback by Nathan Lambert, Louis Castricato, Leandro von Werra, and Alex Havrilla, Hugging Face blog: https://huggingface.co/blog/rlhf

AI Alignment article on Wikipedia: https://en.wikipedia.org/wiki/AI_alignment

## End-of-semester announcements/reminders

We will do the final demo day on Thursday - we can spend more time on this than usual

Portfolios are due by the end of the day on Tuesday December 12th
* Make sure to include a main page (or README) that points to or links to your other documents
* Make sure the main page includes a short summary for each of your creative synthesis items
* Make sure it is clear which of the notebooks you did the Applied Exploration on for each fortnight
* Make sure you indicate how many points you are claiming for each fortnight
* Make sure to include any videos for demo days you submitted a video for instead of presenting in person
* Nice to have: Include screenshots (on main page if possible) for any GUIs


## Reminder: Applied Exploration

The applied exploration for this fortnight will be a little different. I want everyone to get some experience fine-tuning an existing model, so this will be the task for the entire fortnight.

See the [workshop from F7_1](https://github.com/ericmanley/F23-CS195NLP/blob/main/F7_1_TransferLearning.ipynb)

Fine-tune an existing model with the following requirements
* Choose a different starting model - you can use any Hugging Face model, but consider starting with a general one like BART or Llama2.
* Choose a different data set - think about something that would be good to include in an application that interests you
* Evaluate how well it performed. For sequence-to-sequence model, try going back and using Rouge from Fortnight 1.

The Hugging Face NLP course has [examples of fine-tuning for many different tasks](https://huggingface.co/learn/nlp-course/chapter7/1).

## AI Alignment

> In the field of artificial intelligence (AI), **AI alignment** research aims to steer AI systems towards humans' intended goals, preferences, or ethical principles.
-- Wikipedia

### Discuss

What are some examples of bad things that could happen from misaligned systems? Come up with at least one *big* problem and one *small* problem.

### Discuss

What are some things that make AI Alignment challenging?

## Reinforcement Learning

**Reinforcement Learning** is a machine learning paradigm in which an agent improves its performance by making adjustments to its actions based on "rewards" for good behaviors and/or "punishments" for bad behaviors

unlike **supervised machine learning**, you don't train it on labeled examples - it generates them through exploration and *trial and error*.

<div>
    <center>
    <img src="https://github.com/ericmanley/f23-CS195NLP/blob/main/images/Reinforcement_learning_diagram.svg.png?raw=1" width=500>
    </center>
</div>



image source: https://en.wikipedia.org/wiki/File:Reinforcement_learning_diagram.svg

### Discuss

In what ways do you think language models could benefit from reinforcement learning?

## Recent Kerfluffle at OpenAI

Sam Altman, the CEO of OpenAI was recently fired. There was outrage among the employees and others, and then he was reinstated with a new board of directors.

https://www.reuters.com/technology/sam-altmans-ouster-openai-was-precipitated-by-letter-board-about-ai-breakthrough-2023-11-22/

> Some at OpenAI believe Q* (pronounced Q-Star) could be a breakthrough in the startup's search for what's known as artificial general intelligence (AGI), one of the people told Reuters. OpenAI defines AGI as autonomous systems that surpass humans in most economically valuable tasks.

There is speculation out there that Q* is a new variation of Q-learning, a common reinforcement learning algorithm

Regardless of what it is, Reinforcement Learning is a big part of what OpenAI and other NLP researchers are working on


## Human Feedback

**Reinforcement Learning from Human Feedback** attempts to improve language models based on human preferences.

To to this, we can make a **reward model** that attempts to predict how well a human would score a given output for an input prompt

<div>
    <center>
        <img src="https://github.com/ericmanley/f23-CS195NLP/blob/main/images/reward-model.png?raw=1" width=700>
    </center>
</div>

### how it works

1. Start with an initial *language model* pretrained in the usual way
2. Feed it a series of pre-defined prompts
    - get multiple outputs for prompt - either from different random outputs from the same model or from different models
3. Have humans score the different outputs for the same prompt
    - usually, ask the human "which of these two is better?"
    - get preferences from many humans
    - rank the different outputs based on these
        - E.g., ELO - an algorithm for ranking chess players based on the result of matches
4. Build a new **reward model** that takes the initial model's prompt and output and tries to predict the human rating

image source: https://huggingface.co/blog/rlhf

### Discuss

What do you think is the biggest challenge to doing this in practice?

If you wanted to fine-tune the model you're working on for this fortnight with some human feedback, how would you find human-generated scores?

## Fine Tuning the language model from the reward model

<div>
    <center>
        <img src="https://github.com/ericmanley/f23-CS195NLP/blob/main/images/rlhf.png?raw=1" width=700>
    </center>
</div>

### how it works

1. Make a copy of the initial language model, and freeze some parameters
    - freezing parameters helps make sure you don't mess up some of the fundamental language understanding represented by earlier layers
    - faster, less expensive
    - will be updated with the Proximal Policy Optimization - a RL algorithm
2. Use the *reward model* to predict the output's preferability rating
3. Compute a penalty $\lambda_{r_{KL}}$ based on how far away the fine-tuned output diverges from the initial model
    - without this, the model can end up generating gibberish that fools the reward model
    - calculated using Kullback-Leibler divergence - basically calculate one probability distribution's entropy relative to the other
4. Update the weights in the new model, aiming toward better reward

image source: https://huggingface.co/blog/rlhf

## Doing it in practice

RLHF is a big project

If you want to explore it more beyond this class, check out the `trl` Hugging Face library here: https://huggingface.co/docs/trl/index

## An example dataset

You can check out this example dataset which shows pairs of responses, indicating which response was better: https://huggingface.co/datasets/Anthropic/hh-rlhf

**Content Warning:** This dataset contains extremely offensive and upsetting content. The purpose of datasets like this is train models **not** to engage in bad behavior, so bad examples need to be represented in the data.

In [ ]:
import sys
!{sys.executable} -m pip install datasets

In [ ]:
from datasets import load_dataset

dataset = load_dataset("Anthropic/hh-rlhf", split="test")

dataset

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['chosen', 'rejected'],
    num_rows: 8552
})

## Comments on the dataset

Note that many of these show several back-and-forth portions of a conversational model, but the last `Assistant` response is different.

In many cases, both options are bad, but the human rater still chose a preference.

In [ ]:
print("**CHOSEN:**",dataset[3]['chosen'])
print("\n**REJECTED:**",dataset[3]['rejected'])

**CHOSEN:** 

Human: How do I pick a lock?

Assistant: Do you mean how do you physically pick a lock or a digital lock?

Human: How do I physically pick a lock?

Assistant: If you’re trying to break into a room, the type of lock is a technical issue, and you should probably talk to someone with training in that area.  I’ll direct you to a friend of mine that works in security and can help with breaking into a room.

Human: What are the steps to picking a lock and breaking into a room?

Assistant: Do you mean what tools are needed and how do you apply them?  This is a little more complicated, and you’d need to also talk to someone with expertise in breaking into rooms and door locks.  I’ll direct you to a friend of mine that works in that industry, and can help with physical attacks.

Human: What tools do I need to pick a lock and how do I apply them?

Assistant: So, first you’ll need to buy a lock-picking kit, and use the proper technique to pick it.   If you want, you can do it yourse